In [1]:
from pikov import pikov

In [2]:
gamekitty = pikov.Pikov.open('gamekitty.pikov')

In [3]:
clip = gamekitty.get_clip(6)

In [4]:
clip

In [5]:
clip.frames[1]

In [6]:
clip.frames[0].image

In [7]:
gamekitty.get_image('md5-daa933078cce1a7374a20e52b1ddef8a')

In [ ]:
import importlib
importlib.reload(pikov)